In [1]:
!python -V

Python 3.6.3 :: Anaconda custom (64-bit)


# iPython Notebook -- Create Tidy DataSet

- Change the location of your {home} dataset before running this notebook
- The structure of files required in this notebook is similar to this:

    {home}:
      |
      --> well_lat_lng.csv (latitude & longitude coordinates)
      |
      --> {SPE_006_originalData/OilSandsDB}
            |
            --> PICKS_DIC.TXT
            --> PICKS.TXT
            --> WELLS.TXT
            --> {Logs}
                  |
                  --> All logs files (las files)
 

## Import Libraries

In [4]:
import os
import glob
import pandas as pd
import numpy as np
import holoviews as hv
import collections
import matplotlib.pyplot as plt
%matplotlib inline
import lasio
lasio.__version__

'0.19'

## Home directory

In [5]:
home = "./SPE_006_originalData/OilSandsDB"
pd.set_option('display.width', 120)

## Read well locations

In [6]:
# read Site ID (SitID) with lat and long coordinates
locations = pd.read_csv("./well_lat_lng.csv", delimiter=',', dtype=str)
locations = locations[['SitID', 'lng', 'lat']]
locations.head()

,SitID,lng,lat
0,102496,-110.12932,54.785907
1,102497,-110.269446,54.782284
2,102498,-110.186851,54.785892
3,102500,-110.269422,54.829624
4,102501,-110.224832,54.840471


## Read dictionary of Stratigraphic picks

In [7]:
# describe the stratigraphic 'picks' used in Picks
picks_dicts = pd.read_csv(home + "/PICKS_DIC.TXT", delimiter='\t', dtype=str)
picks_dicts.columns = ['HorID', 'Description']
picks_dicts.head()

,HorID,Description
0,1000,mannville
1,2000,t61
2,3000,t51
3,4000,t41
4,5000,t31


## Read stratigraphic picks 

In [8]:
# stratigraphic picks and dropping NaN values
picks = pd.read_csv(home + "/PICKS.TXT", delimiter='\t')
picks['SitID'] = picks['SitID'].astype(str)
picks['HorID'] = picks['HorID'].astype(str)
picks['Pick'] = picks['Pick'].str.strip()
picks['Pick'] = pd.to_numeric(picks['Pick'])
# drop rows with NA values
picks = picks.dropna()
picks.head()

,SitID,HorID,Pick,Quality
0,102496,1000,321.0,1
4,102496,5000,438.0,2
6,102496,7000,458.0,3
7,102496,9000,468.0,3
8,102496,9500,468.0,3


## Read wells with SiteID and UWI

In [9]:
# list of wells
wellst = pd.read_csv(home + "/WELLS.TXT", delimiter= '\t', dtype=str)
wellst.head()

,SitID,UWI (AGS),UWI
0,102496,0674010812000,00/12-08-067-01W4/0
1,102497,0674020807000,00/07-08-067-02W4/0
2,102498,0674021109000,00/09-11-067-02W4/0
3,102500,0674022910000,00/10-29-067-02W4/0
4,102501,0674023406000,00/06-34-067-02W4/0


## Merging Wells, Locations, and Picks by SiteID

In [10]:
# merge wellst, locations, and picks in one dataframe
wl = pd.merge(left=wellst, right=locations, left_on=['SitID'], right_on=['SitID'], how='left')
wlp = pd.merge(left=picks, right=wl, left_on=['SitID'], right_on=['SitID'], how='left')
wlp.columns

Index(['SitID', 'HorID', 'Pick', 'Quality', 'UWI (AGS)', 'UWI', 'lng', 'lat'], dtype='object')

In [11]:
wlp.head()

,SitID,HorID,Pick,Quality,UWI (AGS),UWI,lng,lat
0,102496,1000,321.0,1,0674010812000,00/12-08-067-01W4/0,-110.12932,54.785907
1,102496,5000,438.0,2,0674010812000,00/12-08-067-01W4/0,-110.12932,54.785907
2,102496,7000,458.0,3,0674010812000,00/12-08-067-01W4/0,-110.12932,54.785907
3,102496,9000,468.0,3,0674010812000,00/12-08-067-01W4/0,-110.12932,54.785907
4,102496,9500,468.0,3,0674010812000,00/12-08-067-01W4/0,-110.12932,54.785907


## Read LAS files (display number of files read)

In [12]:
# las files
path_logs = home + "/Logs"
# las = [x for x in os.listdir(path_logs + "/") if x.endswith(".LAS")]
# print("# LAS files: " + str(len(las)))

In [13]:
# # a sample of curves from a log file
# wellname = 'AB-07-12-093-10W4-0'
# lasio.read(path_logs + "/" + wellname + ".LAS").df().head()

## Reading actual LAS Files with lasio library

In [14]:
las = [lasio.read(path_logs + "/" + filename) for filename in os.listdir(path_logs) if filename.endswith(".LAS")]
type(las), type(las[0])

Header section Parameter regexp=~P was not found.


(list, lasio.las.LASFile)

## Display some components of lasio datatype

### Header

In [15]:
# las[2153].header['Well']

### Parameters

In [16]:
# las[2153].header['Parameter']

### Version

In [17]:
# las[2153].header['Version']

### Curves available

In [18]:
# las[2153].header['Curves']

### Type of Curves available

In [19]:
# # type of curves (logs) present in the las file
# las[2153].header['Curves'].keys()

### Location of UWI may vary inside the file - Function to retrieve UWI

In [20]:
# function to retrieve UWI (location of the UWI may vary within the LAS file)
keywds = ['UWI', 'UWI:1', 'UWI:2', 'UWI:3']
def getKeyValue(idx, keywds):
    for keywd in keywds:
        if (keywd) not in las[idx].header["Well"]:
            pass
        else:
            return [keywd, las[idx].header["Well"][keywd].value]

### Counting how many curves exists across all wells

In [21]:
# counting more common curves
curves = []
for i in range(len(las)):
    df_las = las[i].df()
    curves.extend(list(df_las.columns))

### Counting and Ordering unique data set of curves

In [22]:
# ordered set of curves across all logs files
counter = collections.Counter(curves)
counter = collections.OrderedDict(sorted(counter.items()))
print(counter.keys())
print(counter.values())

odict_keys(['CALI', 'COND', 'DELT', 'DENS', 'DPHI', 'DPHI:1', 'DPHI:2', 'DT', 'GR', 'GR:1', 'GR:2', 'IL', 'ILD', 'ILD:1', 'ILD:2', 'ILM', 'LITH', 'LLD', 'LLS', 'NPHI', 'PHID', 'PHIN', 'RESD', 'RHOB', 'RT', 'SFL', 'SFLU', 'SN', 'SNP', 'SP'])
odict_values([783, 3, 98, 4, 1917, 1, 1, 14, 2169, 1, 1, 2, 2154, 1, 1, 6, 1, 2, 1, 2008, 8, 4, 6, 132, 1, 3, 6, 1, 2, 14])


### Selecting common set of curves across all wells

In [23]:
# Selecting only those wells which have common curves: DEPT; DPHI; GR; ILD; NPHI
keys = ['DEPT', 'DPHI', 'DPHI:1', 'DPHI:2', 'GR', 'GR:1', 'GR:2', 'ILD', 'ILD:1', 'ILD:2', \
        'NPHI', 'NPHI:1', 'NPHI:2', 'PHIN', 'PHIN:1', 'PHIN:2']
values = ['DEPT', 'DPHI', 'DPHI', 'DPHI', 'GR', 'GR', 'GR', 'ILD', 'ILD', 'ILD', \
          'NPHI', 'NPHI', 'NPHI', 'NPHI', 'NPHI', 'NPHI']
dictOfCurves = dict(zip(keys, values))

columns = ['i', 'key_uwi', 'uwi', 'curvesNew', 'curvesOld']
rows = []

### Create dataframe with standardized set of curves [GR, DPHI, ILD, DEPT, NPHI]

In [24]:
# create dataframe with standardized set of curves [GR, DPHI, ILD, DEPT, NPHI]
for i in range(len(las)):
    las_curves = las[i].header['Curves'].keys()
    c = [str(dictOfCurves[c]) for c in keys if c in las_curves]
    key_uwi, uwi = getKeyValue(i, keywds)
    rows.append([i, key_uwi, uwi, list(set(c)), las_curves])
    
df = pd.DataFrame(rows, columns=columns)
rows = []
df = df.reset_index(drop=True)

In [25]:
df.head()

,i,key_uwi,uwi,curvesNew,curvesOld
0,0,UWI,00/01-01-073-05W5/0,"[GR, DPHI, NPHI, ILD, DEPT]","[DEPT, DPHI, NPHI, GR, CALI, ILD]"
1,1,UWI,00/01-01-095-19W4/0,"[GR, DPHI, NPHI, ILD, DEPT]","[DEPT, GR, CALI, NPHI, DPHI, ILD]"
2,2,UWI,00/01-03-085-15W400,"[GR, DPHI, NPHI, ILD, DEPT]","[DEPT, DPHI, NPHI, GR, ILD]"
3,3,UWI,00/01-04-075-23W4/0,"[GR, DPHI, NPHI, ILD, DEPT]","[DEPT, ILD, NPHI, DPHI, GR, CALI]"
4,4,UWI,00/01-05-085-15W400,"[GR, DPHI, NPHI, ILD, DEPT]","[DEPT, GR, NPHI, DPHI, ILD]"


In [26]:
len(df)

2171

In [27]:
# drop rows with less than 5 curves including DEPT
dfl = df[df['curvesNew'].map(len) > 4]
dfl = dfl.reset_index(drop=True)

In [28]:
len(dfl)

1848

In [29]:
dfl.head()

,i,key_uwi,uwi,curvesNew,curvesOld
0,0,UWI,00/01-01-073-05W5/0,"[GR, DPHI, NPHI, ILD, DEPT]","[DEPT, DPHI, NPHI, GR, CALI, ILD]"
1,1,UWI,00/01-01-095-19W4/0,"[GR, DPHI, NPHI, ILD, DEPT]","[DEPT, GR, CALI, NPHI, DPHI, ILD]"
2,2,UWI,00/01-03-085-15W400,"[GR, DPHI, NPHI, ILD, DEPT]","[DEPT, DPHI, NPHI, GR, ILD]"
3,3,UWI,00/01-04-075-23W4/0,"[GR, DPHI, NPHI, ILD, DEPT]","[DEPT, ILD, NPHI, DPHI, GR, CALI]"
4,4,UWI,00/01-05-085-15W400,"[GR, DPHI, NPHI, ILD, DEPT]","[DEPT, GR, NPHI, DPHI, ILD]"


In [30]:
# final dataframe with wells having these curves [DEPT, DPHI, GR, ILD, NPHI]
dl = pd.DataFrame([], columns=['UWI', 'DEPT', 'DPHI', 'GR', 'ILD', 'NPHI'])
dl.columns

for i in range(len(dfl)):
    well_num = dfl.iloc[i]['i']
    uwi_type = dfl.iloc[i]['key_uwi']
    uwi = las[well_num].header['Well'][uwi_type].value

    nrecords = len(las[well_num].header['Curves']['DEPT'].data)

    UWI = pd.Series([uwi] * nrecords)
    DEPT = pd.Series(las[well_num].header['Curves']['DEPT'].data)
    DPHI = pd.Series(las[well_num].header['Curves']['DPHI'].data)
    GR = pd.Series(las[well_num].header['Curves']['GR'].data)
    ILD = pd.Series(las[well_num].header['Curves']['ILD'].data)
    NPHI = pd.Series(las[well_num].header['Curves']['NPHI'].data)

    d = {
        'UWI': UWI,
        'DEPT': DEPT,
        'DPHI': DPHI,
        'GR': GR,
        'ILD': ILD,
        'NPHI': NPHI
    }
    dl = dl.append(pd.DataFrame(d), ignore_index=True)

In [31]:
dl.head()

,DEPT,DPHI,GR,ILD,NPHI,UWI
0,390.00,0.199,82.478,2.923,0.457,00/01-01-073-05W5/0
1,390.25,0.208,86.413,2.925,0.456,00/01-01-073-05W5/0
2,390.50,0.246,90.229,2.917,0.452,00/01-01-073-05W5/0
3,390.75,0.266,90.944,2.898,0.475,00/01-01-073-05W5/0
4,391.00,0.287,88.866,2.890,0.484,00/01-01-073-05W5/0


In [32]:
# get UWI (AGS) using a not-well-formed  UWI
def getUWIAGS(wrongUWI):

    s1 = wrongUWI[9:12]
    s2 = wrongUWI[16:17]
    s3 = wrongUWI[13:15]
    s4 = wrongUWI[6:8]
    s5 = wrongUWI[3:5]
    s6 = wrongUWI[0:2]
    s7 = wrongUWI[18:19]
    
    return(s1 + s2 + s3 + s4 + s5 + s6 + s7)

In [33]:
# getUWIAGS('AD/05-15-088-08W400')

In [34]:
# len(dl)

In [35]:
# (dl.iloc[1474750].UWI, getUWIAGS(dl.iloc[1474750].UWI))

In [36]:
# adding UWI_AGS column to join table Wellst which has the SitID. SitID is common to all data tables
dl['UWI_AGS'] = [getUWIAGS(dl.iloc[i].UWI) for i in range(len(dl))]

In [37]:
dl.head()

,DEPT,DPHI,GR,ILD,NPHI,UWI,UWI_AGS
0,390.00,0.199,82.478,2.923,0.457,00/01-01-073-05W5/0,0735050101000
1,390.25,0.208,86.413,2.925,0.456,00/01-01-073-05W5/0,0735050101000
2,390.50,0.246,90.229,2.917,0.452,00/01-01-073-05W5/0,0735050101000
3,390.75,0.266,90.944,2.898,0.475,00/01-01-073-05W5/0,0735050101000
4,391.00,0.287,88.866,2.890,0.484,00/01-01-073-05W5/0,0735050101000


In [38]:
# see how this 'not-well-formed UWI': AD/05-15-088-08W400, should be this: AD/05-15-088-08W4/0
dl.tail()

,DEPT,DPHI,GR,ILD,NPHI,UWI,UWI_AGS
1474746,142.00,0.039,64.370,50.572,0.149,AD/05-15-088-08W400,0884081505AD0
1474747,142.25,0.034,64.406,51.559,0.152,AD/05-15-088-08W400,0884081505AD0
1474748,142.50,0.032,60.770,56.706,0.156,AD/05-15-088-08W400,0884081505AD0
1474749,142.75,0.039,53.479,61.181,0.148,AD/05-15-088-08W400,0884081505AD0
1474750,143.00,0.030,48.631,61.220,0.150,AD/05-15-088-08W400,0884081505AD0


### Merging dataframe 'dl' with logs and wells table (wellst) to get SitID

In [39]:
# merging dataframe 'dl' with logs and wells table (wellst) to get SitID  
dl = pd.merge(dl, wellst[['SitID', 'UWI (AGS)']], left_on='UWI_AGS', right_on='UWI (AGS)', how='left')
dt = dl[['SitID', 'DEPT', 'DPHI', 'GR', 'ILD', 'NPHI']]
dt.head()

,SitID,DEPT,DPHI,GR,ILD,NPHI
0,142218,390.00,0.199,82.478,2.923,0.457
1,142218,390.25,0.208,86.413,2.925,0.456
2,142218,390.50,0.246,90.229,2.917,0.452
3,142218,390.75,0.266,90.944,2.898,0.475
4,142218,391.00,0.287,88.866,2.890,0.484


In [40]:
len(dt)

1474751

In [41]:
dt.tail()

,SitID,DEPT,DPHI,GR,ILD,NPHI
1474746,117119,142.00,0.039,64.370,50.572,0.149
1474747,117119,142.25,0.034,64.406,51.559,0.152
1474748,117119,142.50,0.032,60.770,56.706,0.156
1474749,117119,142.75,0.039,53.479,61.181,0.148
1474750,117119,143.00,0.030,48.631,61.220,0.150


In [42]:
dt.index

Int64Index([      0,       1,       2,       3,       4,       5,       6,       7,       8,       9,
            ...
            1474741, 1474742, 1474743, 1474744, 1474745, 1474746, 1474747, 1474748, 1474749, 1474750],
           dtype='int64', length=1474751)

### Merging dataframe 'dt' with 'wlp' on SitID and [DEPT/Pick]

In [43]:
# merging dataframe 'dt' with 'wlp' on SitID and [DEPT/Pick]
m = pd.merge(left=dt, right=wlp, left_on=['SitID', 'DEPT'], right_on=['SitID', 'Pick'], how='left')
# change HorID to string
m['HorID'] = m['HorID'].astype(str)
# drop NA values
m = m.dropna()
m.head()

,SitID,DEPT,DPHI,GR,ILD,NPHI,HorID,Pick,Quality,UWI (AGS),UWI,lng,lat
12,142218,393.0,0.299,72.249,3.338,0.429,1000,393.0,1.0,0735050101000,00/01-01-073-05W5/0,-114.620656,55.288691
636,142218,549.0,0.247,76.644,4.313,0.427,4000,549.0,1.0,0735050101000,00/01-01-073-05W5/0,-114.620656,55.288691
776,142218,584.0,0.184,72.472,5.379,0.357,5000,584.0,1.0,0735050101000,00/01-01-073-05W5/0,-114.620656,55.288691
828,142218,597.0,0.180,69.942,4.265,0.367,7000,597.0,1.0,0735050101000,00/01-01-073-05W5/0,-114.620656,55.288691
868,142218,607.0,0.191,57.457,9.279,0.312,11000,607.0,1.0,0735050101000,00/01-01-073-05W5/0,-114.620656,55.288691


In [44]:
len(m)

10220

### Saving Final dataset for ML Modeling

In [45]:
m = m.reset_index(drop=True)
m['Id'] = m.index.values
m = m[['Id', 'SitID', 'DPHI', 'GR', 'ILD', 'NPHI', 'lng', 'lat', 'HorID', 'Pick']]
# m.to_csv('m_out.csv', sep="\t", index=None)

### Adding 5 obervations before & after to every single observation at the DEPT indicated

In [46]:
# reading data file with 5 observation before and after. Prepared in R by Arun Atmacharan
obs_befr_aftr = pd.read_csv("./obs_reduced.csv", delimiter=',')

In [47]:
obs_befr_aftr.head()

,SitIDr,DEPTr,HorIDr,Pickr,DPHIr1,GRr1,ILDr1,NPHIr1,DPHIr2,GRr2,...,ILDr8,NPHIr8,DPHIr9,GRr9,ILDr9,NPHIr9,DPHIr10,GRr10,ILDr10,NPHIr10
0,142218,393.0,1000,393.0,0.215,88.403,2.725,0.471,0.190,91.038,...,4.097,0.355,0.374,54.888,3.848,0.481,0.454,61.614,3.267,0.536
1,142218,549.0,4000,549.0,0.226,78.224,4.498,0.399,0.239,78.255,...,4.750,0.410,0.229,55.811,5.105,0.438,0.201,56.461,5.621,0.296
2,142218,584.0,5000,584.0,0.210,82.798,4.723,0.390,0.197,83.507,...,6.752,0.330,0.120,70.858,7.297,0.326,0.132,74.797,7.740,0.308
3,142218,597.0,7000,597.0,0.528,78.466,1.863,0.513,0.483,77.323,...,6.046,0.403,0.280,74.096,6.132,0.399,0.221,74.690,6.103,0.372
4,142218,607.0,11000,607.0,0.150,93.824,5.979,0.365,0.137,90.261,...,10.898,0.312,0.204,46.169,11.922,0.308,0.199,41.485,11.464,0.302


In [48]:
obs_befr_aftr.columns.to_series().groupby(obs_befr_aftr.dtypes).groups

{dtype('int64'): Index(['SitIDr', 'HorIDr'], dtype='object'),
 dtype('float64'): Index(['DEPTr', 'Pickr', 'DPHIr1', 'GRr1', 'ILDr1', 'NPHIr1', 'DPHIr2', 'GRr2', 'ILDr2', 'NPHIr2', 'DPHIr3', 'GRr3',
        'ILDr3', 'NPHIr3', 'DPHIr4', 'GRr4', 'ILDr4', 'NPHIr4', 'DPHIr5', 'GRr5', 'ILDr5', 'NPHIr5', 'DPHIr6', 'GRr6',
        'ILDr6', 'NPHIr6', 'DPHIr7', 'GRr7', 'ILDr7', 'NPHIr7', 'DPHIr8', 'GRr8', 'ILDr8', 'NPHIr8', 'DPHIr9', 'GRr9',
        'ILDr9', 'NPHIr9', 'DPHIr10', 'GRr10', 'ILDr10', 'NPHIr10'],
       dtype='object')}

In [50]:
obs_befr_aftr.SitIDr = obs_befr_aftr.SitIDr.astype(str)
obs_befr_aftr.columns.to_series().groupby(obs_befr_aftr.dtypes).groups

{dtype('int64'): Index(['HorIDr'], dtype='object'),
 dtype('float64'): Index(['DEPTr', 'Pickr', 'DPHIr1', 'GRr1', 'ILDr1', 'NPHIr1', 'DPHIr2', 'GRr2', 'ILDr2', 'NPHIr2', 'DPHIr3', 'GRr3',
        'ILDr3', 'NPHIr3', 'DPHIr4', 'GRr4', 'ILDr4', 'NPHIr4', 'DPHIr5', 'GRr5', 'ILDr5', 'NPHIr5', 'DPHIr6', 'GRr6',
        'ILDr6', 'NPHIr6', 'DPHIr7', 'GRr7', 'ILDr7', 'NPHIr7', 'DPHIr8', 'GRr8', 'ILDr8', 'NPHIr8', 'DPHIr9', 'GRr9',
        'ILDr9', 'NPHIr9', 'DPHIr10', 'GRr10', 'ILDr10', 'NPHIr10'],
       dtype='object'),
 dtype('O'): Index(['SitIDr'], dtype='object')}

In [51]:
m.head()

,Id,SitID,DPHI,GR,ILD,NPHI,lng,lat,HorID,Pick
0,0,142218,0.299,72.249,3.338,0.429,-114.620656,55.288691,1000,393.0
1,1,142218,0.247,76.644,4.313,0.427,-114.620656,55.288691,4000,549.0
2,2,142218,0.184,72.472,5.379,0.357,-114.620656,55.288691,5000,584.0
3,3,142218,0.180,69.942,4.265,0.367,-114.620656,55.288691,7000,597.0
4,4,142218,0.191,57.457,9.279,0.312,-114.620656,55.288691,11000,607.0


In [52]:
m.columns.to_series().groupby(m.dtypes).groups

{dtype('int64'): Index(['Id'], dtype='object'),
 dtype('float64'): Index(['DPHI', 'GR', 'ILD', 'NPHI', 'Pick'], dtype='object'),
 dtype('O'): Index(['SitID', 'lng', 'lat', 'HorID'], dtype='object')}

In [63]:
# merging dataframe 'm' with 'obs_befr_aftr' on SitID and [DEPT/Pick]
ds = pd.merge(left=m, right=obs_befr_aftr, left_on=['SitID', 'Pick'], right_on=['SitIDr', 'DEPTr'], how='left')

In [64]:
len(ds)

16830

In [65]:
ds = ds.dropna() # to be sure we don't get null values or incomplete observations

In [66]:
len(ds)

16829

In [67]:
ds.head()

,Id,SitID,DPHI,GR,ILD,NPHI,lng,lat,HorID,Pick,...,ILDr8,NPHIr8,DPHIr9,GRr9,ILDr9,NPHIr9,DPHIr10,GRr10,ILDr10,NPHIr10
0,0,142218,0.299,72.249,3.338,0.429,-114.620656,55.288691,1000,393.0,...,4.097,0.355,0.374,54.888,3.848,0.481,0.454,61.614,3.267,0.536
1,1,142218,0.247,76.644,4.313,0.427,-114.620656,55.288691,4000,549.0,...,4.750,0.410,0.229,55.811,5.105,0.438,0.201,56.461,5.621,0.296
2,2,142218,0.184,72.472,5.379,0.357,-114.620656,55.288691,5000,584.0,...,6.752,0.330,0.120,70.858,7.297,0.326,0.132,74.797,7.740,0.308
3,3,142218,0.180,69.942,4.265,0.367,-114.620656,55.288691,7000,597.0,...,6.046,0.403,0.280,74.096,6.132,0.399,0.221,74.690,6.103,0.372
4,4,142218,0.191,57.457,9.279,0.312,-114.620656,55.288691,11000,607.0,...,10.898,0.312,0.204,46.169,11.922,0.308,0.199,41.485,11.464,0.302


In [68]:
ds.columns

Index(['Id', 'SitID', 'DPHI', 'GR', 'ILD', 'NPHI', 'lng', 'lat', 'HorID', 'Pick', 'SitIDr', 'DEPTr', 'HorIDr', 'Pickr',
       'DPHIr1', 'GRr1', 'ILDr1', 'NPHIr1', 'DPHIr2', 'GRr2', 'ILDr2', 'NPHIr2', 'DPHIr3', 'GRr3', 'ILDr3', 'NPHIr3',
       'DPHIr4', 'GRr4', 'ILDr4', 'NPHIr4', 'DPHIr5', 'GRr5', 'ILDr5', 'NPHIr5', 'DPHIr6', 'GRr6', 'ILDr6', 'NPHIr6',
       'DPHIr7', 'GRr7', 'ILDr7', 'NPHIr7', 'DPHIr8', 'GRr8', 'ILDr8', 'NPHIr8', 'DPHIr9', 'GRr9', 'ILDr9', 'NPHIr9',
       'DPHIr10', 'GRr10', 'ILDr10', 'NPHIr10'],
      dtype='object')

In [69]:
ds = ds[['SitID', 'HorID', 'lng', 'lat', \
         'DPHI', 'GR', 'ILD', 'NPHI', \
         'DPHIr1', 'GRr1', 'ILDr1', 'NPHIr1', \
         'DPHIr2', 'GRr2', 'ILDr2', 'NPHIr2', \
         'DPHIr3', 'GRr3', 'ILDr3', 'NPHIr3', \
         'DPHIr4', 'GRr4', 'ILDr4', 'NPHIr4', \
         'DPHIr5', 'GRr5', 'ILDr5', 'NPHIr5', \
         'DPHIr6', 'GRr6', 'ILDr6', 'NPHIr6', \
         'DPHIr7', 'GRr7', 'ILDr7', 'NPHIr7', \
         'DPHIr8', 'GRr8', 'ILDr8', 'NPHIr8', \
         'DPHIr9', 'GRr9', 'ILDr9', 'NPHIr9', \
         'DPHIr10', 'GRr10', 'ILDr10', 'NPHIr10', \
         'Pick']]
ds.head()

,SitID,HorID,lng,lat,DPHI,GR,ILD,NPHI,DPHIr1,GRr1,...,NPHIr8,DPHIr9,GRr9,ILDr9,NPHIr9,DPHIr10,GRr10,ILDr10,NPHIr10,Pick
0,142218,1000,-114.620656,55.288691,0.299,72.249,3.338,0.429,0.215,88.403,...,0.355,0.374,54.888,3.848,0.481,0.454,61.614,3.267,0.536,393.0
1,142218,4000,-114.620656,55.288691,0.247,76.644,4.313,0.427,0.226,78.224,...,0.410,0.229,55.811,5.105,0.438,0.201,56.461,5.621,0.296,549.0
2,142218,5000,-114.620656,55.288691,0.184,72.472,5.379,0.357,0.210,82.798,...,0.330,0.120,70.858,7.297,0.326,0.132,74.797,7.740,0.308,584.0
3,142218,7000,-114.620656,55.288691,0.180,69.942,4.265,0.367,0.528,78.466,...,0.403,0.280,74.096,6.132,0.399,0.221,74.690,6.103,0.372,597.0
4,142218,11000,-114.620656,55.288691,0.191,57.457,9.279,0.312,0.150,93.824,...,0.312,0.204,46.169,11.922,0.308,0.199,41.485,11.464,0.302,607.0


In [70]:
# occurrences of every HorID
ds.HorID.value_counts()

13000    2655
12000    2580
11000    2394
9500     1750
9000     1729
5000     1057
10000    1048
14000     999
7000      862
1000      857
4000      252
6000      251
3000      205
2000      190
Name: HorID, dtype: int64

In [78]:
# ds['id'] = ds.index
cols = ['id', 'SitID', 'HorID', 'lng', 'lat', 'DPHI', 'GR', 'ILD', 'NPHI',
       'DPHIr1', 'GRr1', 'ILDr1', 'NPHIr1', 'DPHIr2', 'GRr2', 'ILDr2',
       'NPHIr2', 'DPHIr3', 'GRr3', 'ILDr3', 'NPHIr3', 'DPHIr4', 'GRr4',
       'ILDr4', 'NPHIr4', 'DPHIr5', 'GRr5', 'ILDr5', 'NPHIr5', 'DPHIr6',
       'GRr6', 'ILDr6', 'NPHIr6', 'DPHIr7', 'GRr7', 'ILDr7', 'NPHIr7',
       'DPHIr8', 'GRr8', 'ILDr8', 'NPHIr8', 'DPHIr9', 'GRr9', 'ILDr9',
       'NPHIr9', 'DPHIr10', 'GRr10', 'ILDr10', 'NPHIr10', 'Pick']
ds = ds[['id', 'SitID', 'HorID', 'lng', 'lat', 'DPHI', 'GR', 'ILD', 'NPHI',
       'DPHIr1', 'GRr1', 'ILDr1', 'NPHIr1', 'DPHIr2', 'GRr2', 'ILDr2',
       'NPHIr2', 'DPHIr3', 'GRr3', 'ILDr3', 'NPHIr3', 'DPHIr4', 'GRr4',
       'ILDr4', 'NPHIr4', 'DPHIr5', 'GRr5', 'ILDr5', 'NPHIr5', 'DPHIr6',
       'GRr6', 'ILDr6', 'NPHIr6', 'DPHIr7', 'GRr7', 'ILDr7', 'NPHIr7',
       'DPHIr8', 'GRr8', 'ILDr8', 'NPHIr8', 'DPHIr9', 'GRr9', 'ILDr9',
       'NPHIr9', 'DPHIr10', 'GRr10', 'ILDr10', 'NPHIr10', 'Pick']]

In [79]:
ds.head()

,id,SitID,HorID,lng,lat,DPHI,GR,ILD,NPHI,DPHIr1,...,NPHIr8,DPHIr9,GRr9,ILDr9,NPHIr9,DPHIr10,GRr10,ILDr10,NPHIr10,Pick
0,0,142218,1000,-114.620656,55.288691,0.299,72.249,3.338,0.429,0.215,...,0.355,0.374,54.888,3.848,0.481,0.454,61.614,3.267,0.536,393.0
1,1,142218,4000,-114.620656,55.288691,0.247,76.644,4.313,0.427,0.226,...,0.410,0.229,55.811,5.105,0.438,0.201,56.461,5.621,0.296,549.0
2,2,142218,5000,-114.620656,55.288691,0.184,72.472,5.379,0.357,0.210,...,0.330,0.120,70.858,7.297,0.326,0.132,74.797,7.740,0.308,584.0
3,3,142218,7000,-114.620656,55.288691,0.180,69.942,4.265,0.367,0.528,...,0.403,0.280,74.096,6.132,0.399,0.221,74.690,6.103,0.372,597.0
4,4,142218,11000,-114.620656,55.288691,0.191,57.457,9.279,0.312,0.150,...,0.312,0.204,46.169,11.922,0.308,0.199,41.485,11.464,0.302,607.0


In [80]:
# writing final dataset to disk - ready for NIA
ds.to_csv('ds_out_ready.csv', sep="\t", index=None)

### end of Create tidy dataset